In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from vae_audio import VAE,train
# カスタムデータセット
class AudioDataset(Dataset):
    def __init__(self, txt_folder_path):
        self.data = []
        for txt_file in os.listdir(txt_folder_path):
            if txt_file.endswith(".txt"):
                with open(os.path.join(txt_folder_path, txt_file), 'r') as f:
                    lines = f.readlines()
                    video_data = []
                    for line in lines:
                        video_data.extend(list(map(float, line.strip().split())))
                    self.data.append(video_data)
        self.data = np.array(self.data)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        sample_reshaped = np.reshape(sample, (60, 300))
        return torch.tensor(sample_reshaped, dtype=torch.float32)


# パスを設定してデータセットとデータローダを作成
txt_folder_path = "/home/zhangzehang2/emotion/audio-txt4/Male"
dataset1 = AudioDataset(txt_folder_path)
dataloader1 = DataLoader(dataset1, batch_size=10, shuffle=True)

# パスを設定してデータセットとデータローダを作成
txt_folder_path = "/home/zhangzehang2/emotion/audio-txt4/Female"
dataset2 = AudioDataset(txt_folder_path)
dataloader2 = DataLoader(dataset2, batch_size=10, shuffle=True)

In [3]:
# 既に提供されているコードの上で以下を追加/変更

# VAEのインスタンスを作成
latent_dim = 8
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE(latent_dim, device).to(device)

# トレーニングのハイパーパラメータを設定
class Args:
    latent_dim = latent_dim
    batch_size = 10
    variational_beta = 1

args = Args()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# トレーニングのエポック数
num_epochs = 20

print("Male-audio-vae-loss")
for epoch in range(num_epochs):
    average_loss, _, _ = train(args, model, dataloader1, optimizer, device=device)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")

print("Femle-audio-vae-loss")
for epoch in range(num_epochs):
    average_loss, _, _ = train(args, model, dataloader2, optimizer, device=device)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")

Male-audio-vae-loss
Epoch 1/10, Loss: 0.9891
Epoch 2/10, Loss: 0.9775
Epoch 3/10, Loss: 0.9768
Epoch 4/10, Loss: 0.9760
Epoch 5/10, Loss: 0.9758
Epoch 6/10, Loss: 0.9756
Epoch 7/10, Loss: 0.9756
Epoch 8/10, Loss: 0.9756
Epoch 9/10, Loss: 0.9755
Epoch 10/10, Loss: 0.9755
Femle-audio-vae-loss
Epoch 1/10, Loss: 1.1959
Epoch 2/10, Loss: 1.1930
Epoch 3/10, Loss: 1.1957
Epoch 4/10, Loss: 1.1930
Epoch 5/10, Loss: 1.1929
Epoch 6/10, Loss: 1.1929
Epoch 7/10, Loss: 1.1929
Epoch 8/10, Loss: 1.1929
Epoch 9/10, Loss: 1.1929
Epoch 10/10, Loss: 1.1928
